In [114]:
import getpass
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
from arcgis.mapping import WebMap
import requests
import os
import time

print('Modules Loaded')

Modules Loaded


In [128]:
# Login to ArcGIS Online (AGOL) by requesting the user to input a username and password
# documentation on getpass: https://docs.python.org/3/library/getpass.html

def login_AGOL():
    try:
        # prompt the user to enter their AGOL username
        username = input("Enter your ArcGIS Online username:")

        # prompt the user to enter their AGOL password (without echoing)
        password = getpass.getpass("Enter your ArcGIS Online password:")

        # create a GIS object and login to AGOL
        gis = GIS("https://www.arcgis.com", username, password)

        # confirm the login was successful
        if gis.users.me is not None:
            print ("Logged in as:", gis.users.me.username)
        else:
            print ("Log in failed. Please check credientials.")
    
        # return the object
        return gis
    
    except Exception as e:
        print ("An error occurred during login:", e)
        return None
    
# run the login function

gis = login_AGOL()

if gis:
    # confirm the user login
    print ("Logged in successfully.")
else:
    print ("Failed to login.")


Enter your ArcGIS Online username:Stacey_Vernon_GES23
Enter your ArcGIS Online password:········
Logged in as: Stacey_Vernon_GES23
Logged in successfully.


In [125]:
# Utilise the AGOL API and the a "static" GeoJSON file to publish data to a feature service. Specifically publish
# Dublin Bike zone data.
# documentation on using the ArcGIS API to create a feature service from GeoJSON here: https://developers.arcgis.com/python/guide/import-data/
# We will determine if a feature layer exists, if not we will create one but if it does - we skip this function

BikeStationsZone_data = None

# generate a list of the below search conditions in our AGOL Item Content
zoneSearch = gis.content.search("title:BikeStations_Zones",item_type="Feature Layer")

# if list is empty, create a new feature layer
if not zoneSearch:

# set up the feature service properties in AGOL for the stations and zone dataset
    item_properties = {
    "title": "BikeStations_Zones", #provide a title for your item
    "description": "Dublin Bike Stations and Zones generated from GeoJSON provided by Bleeper Bikes", #add a description of the data
    "tags": "dublinBikes", # add tags to data to improve AGOL search functionality
    "type": "GeoJson" # define the type of data you are trying to pubish
    }

    # URL link to the GeoJSON file
    geojson_url = "https://data.smartdublin.ie/dataset/09870e46-26a3-4dc2-b632-4d1fba5092f9/resource/40a718a8-cb99-468d-962b-af4fed4b0def/download/bleeperbike_map.geojson"
    geojson_item = gis.content.add(item_properties, geojson_url)

    # publish the GeoJSON as a feature layer
    BikeStationsZone_data = geojson_item.publish()
    print("Data published. New feature layer created.")
    
else:
    
    # if the feature layer already exists, handle it
    BikeStationsZone_data = zoneSearch[0]
    print("Bike Station and Zone data already exists in AGOL.")
    
    
### NB seems to be an issue if the service is in your recycle bin 

Bike Station and Zone data already exists in AGOL.


In [117]:
# Dublin Bike locations are available as a live dataset, updated every 5 minutes
# pull in the Dublin Bike locations from the API

# URL to API endpoint for bike locations
bike_url = "https://data.smartdublin.ie/bleeperbike-api/bikes/bleeper_bikes/current/bikes.geojson"

status = False
while status != True: 
    response = requests.get(bike_url)
    # if the request was successful (status code 200), get the data
    if response.status_code in [200,201]:
        # parse the JSON response
        bikeData = response.json()
        status = True
        print ('Successfully retrieved data')
    else:
        status = False
        time.sleep(60)
        print ('Failed to retrieve data, trying again.')    

Successfully retrieved data


In [118]:
# create a FLC from the GeoJSON

# set up the feature service properties in AGOL for the stations and zone dataset
item_properties = {
    "title": "Bike_Locations", #provide a title for your item
    "description": "Dublin Bike current locations", #add a description of the data
    "tags": "dublinBikes", # add tags to data to improve AGOL search functionality
    "type": "GeoJson" # define the type of data you are trying to pubish
    }
    
# search AGOL contents to check if the layer already exists
fl_search = gis.content.search("title:Bike_Locations",item_type="Feature Layer")

if not fl_search:
    # if search list is empty (meaning a feature layer with that title does not exist), upload the GeoJSON 
    # & publish as a new feature service
    geojson_item = gis.content.add(item_properties, bike_url)
    live_bike_fl = geojson_item.publish()
    print("Successfully created new feature service and published live bike location")
    
else: 
    # if search list is NOT empty (meaning feature layer with that title already exists), overwrite the data within the FL
    live_data_fl = fl_search[0]
    content = FeatureLayerCollection.fromitem(live_data_fl)
    overwrite_result = content.manager.overwrite(bike_url)
    
    # overwrite_result will have {"success":True} or {"success":False}
    if not overwrite_result['success']:
        # if not true
        print("Failed to overwrite Feature Layer Collection with new GeoJSON Data")
    else:
        # if true
        print("Successfully updated Feature Layer Collection with new GeoJSON Data")

Successfully updated Feature Layer Collection with new GeoJSON Data


In [119]:
#  Search AGOL content to see if a map has already been created

map_title = "DublinBike_LocationMap"
existing_maps = gis.content.search(query=f"title:'{map_title}'", item_type = "Web Map")

if existing_maps:
    # retrieve the existing web map item
    web_map_item = existing_maps[0]
    web_map = WebMap(web_map_item)
    print ("Map already exists within AGOL Content")

else:
    print("No existing map found")

Map already exists within AGOL Content


In [122]:
if existing_maps:
    # Ask user if they want to update the map
    user_input = input("Do you wish to update the map? (Y/N): ").strip().upper()
    
    if user_input == 'Y':
        # update the map with layers defined below
        web_map.add_layer(layer=live_bike_fl)
        web_map.add_layer(BikeStationsZone_data)
        web_map.update()
        
        print("Web map updated successfully")
        
    else:
        # user has no updates so construct a URL to access the map
        web_map_url = f"https://www.arcgis.com/home/webmap/viewer.html?webmap={web_map_item.id}"
        print(f"Displaying existing map URL: {web_map_url}")

else: 
    # if map doesn't exist, create one
    web_map = WebMap()
    web_map.add_layer(layer=live_bike_fl)
    web_map.add_layer(BikeStationsZone_data)
    web_map_properties = {
        "title": map_title,
        "description": "A web map of live Dublin Bike locations and their zones/stations obtained from Smart Dublin API",
        "tags": "Dublin Bike, locations, live webmap",
        "snippet": "A webmap of Dublin Bike locations"
    }
    
    web_map.save(item_properties=web_map_properties)
    print ("New webmap created and saved")

Do you wish to update the map? (Y/N): y
Web map updated successfully


In [123]:
# create and display webmap URL
web_map_url = f"https://www.arcgis.com/home/webmap/viewer.html?webmap={web_map_item.id}"
print(f"URL for AGOL Map: {web_map_url}")

#display map within notebook (if supported)
        
try:
    display(web_map)

except ImportError:
    print("Display functionality not available in this environment.")

URL for AGOL Map: https://www.arcgis.com/home/webmap/viewer.html?webmap=626eed6129674ed3b36d545af0c282eb


MapView(hide_mode_switch=True, jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=T…

In [ ]:
# need to do:
# improve commenting 
# improve webmap - symbology?
# product heatmap of current locations - deduce busy areas??